In [4]:
import sys
sys.path.append('../droid_slam')

from tqdm import tqdm
import numpy as np
import torch
import lietorch
import cv2
import os
import glob 
import time
import argparse

import torch.nn.functional as F
from droid import Droid

import matplotlib.pyplot as plt

In [5]:
import evo
from evo.core.trajectory import PoseTrajectory3D
from evo.tools import file_interface
from evo.core import sync
import evo.main_ape as main_ape
from evo.core.metrics import PoseRelation

**Rough**:

In [6]:
delta = torch.randn(1, 36, 43, 70, 2)
delta2 = torch.randn(1, 36, 43, 70, 2)
print((delta-delta2).norm(dim=-1).shape)

torch.Size([1, 36, 43, 70])


In [3]:



def show_image(image):
    image = image.permute(1, 2, 0).cpu().numpy()
    cv2.imshow('image', image / 255.0)
    cv2.waitKey(1)

def print_minmax(arr,desc):
    """visualize depths and uncertainty of any method"""
    
    print("*" * 60)
    print("***{}***  :".format(desc))
    print("arr.shape = {}".format(arr.shape))
    print("type(arr[0,0] = {}".format(type(arr[0,0])))
    print("np.min = {}".format(np.min(arr)))
    print("np.max = {}".format(np.max(arr)))
    print("np.mean = {}".format(np.mean(arr)))
    print("np.median = {}".format(np.median(arr)))
    #print("arr[200:220,200:220] = \n",arr[200:220,200:220])
    print("arr[0:10,0:10] = \n",arr[0:10,0:10])
    print("*" * 60 + "\n")

def image_stream(datapath, use_depth=False, stride=1, use_pred_depth = False, pixelformer_depth_folder = 'pixelformer_depth'):
    """ image generator """

    fx, fy, cx, cy = np.loadtxt(os.path.join(datapath, 'calibration.txt')).tolist()
    image_list = sorted(glob.glob(os.path.join(datapath, 'rgb', '*.png')))[::stride]
    if use_pred_depth:
        depth_list = sorted(glob.glob(os.path.join(datapath, pixelformer_depth_folder, '*.png')))[::stride]
    else:    
        depth_list = sorted(glob.glob(os.path.join(datapath, 'depth', '*.png')))[::stride]
    print(len(image_list), len(depth_list))
    for t, (image_file, depth_file) in enumerate(zip(image_list, depth_list)):
        #print("t = ",t)    
        image = cv2.imread(image_file)
        
        height,width,_= image.shape
        image = image[height%32:, width%32:,:]
        image = cv2.resize(image, (width, height))

        if use_pred_depth:
            depth = cv2.imread(depth_file, -1) / 1000.0
            # if t==0:
            #     print("initialising depth with 0")
            #depth = depth * 0.
            #print_minmax(depth,"pred_depth")
            #sys.exit(0)
        else:
            depth = cv2.imread(depth_file, -1) / 5000.0  # cv2.IMREAD_ANYDEPTH
            #print_minmax(depth,"gt_depth")
            #sys.exit(0)
            

        h0, w0, _ = image.shape
        h1 = int(h0 * np.sqrt((384 * 512) / (h0 * w0)))
        w1 = int(w0 * np.sqrt((384 * 512) / (h0 * w0)))

        image = cv2.resize(image, (w1, h1))
        image = image[:h1-h1%8, :w1-w1%8]
        image = torch.as_tensor(image).permute(2, 0, 1)
        
        depth = torch.as_tensor(depth)
        depth = F.interpolate(depth[None,None], (h1, w1)).squeeze()
        depth = depth[:h1-h1%8, :w1-w1%8]

        intrinsics = torch.as_tensor([fx, fy, cx, cy])
        intrinsics[0::2] *= (w1 / w0)
        intrinsics[1::2] *= (h1 / h0)

        if use_depth:
            yield t, image[None], depth, intrinsics

        else:
            yield t, image[None], intrinsics

In [4]:
class Args:
    def __init__(self):
        self.datapath = "../datasets/ETH3D/train/plant_scene_1"
        self.weights = "../droid.pth"
        self.buffer = 1024
        self.image_size = [240,320]
        self.disable_vis = True
        
        self.beta = 0.5
        self.filter_thresh = 2.0
        self.warmup = 8
        self.keyframe_thresh = 3.5
        self.frontend_thresh = 16.0
        self.frontend_window = 16
        self.frontend_radius = 1
        self.frontend_nms = 0
    
        self.stereo = False
        self.depth = False
    
        self.backend_thresh = 22.0
        self.backend_radius = 2
        self.backend_nms = 3
        self.upsample = False
        self.reconstruction_path = "../saved_reconstruction"
        self.without_depth = False
        self.pixelformer = False

args = Args()
torch.multiprocessing.set_start_method('spawn',force=True)
stride = 1
print("Running evaluation on {}".format(args.datapath))
print(vars(args)) # or print(args.__dict__)


Running evaluation on ../datasets/ETH3D/train/plant_scene_1
{'datapath': '../datasets/ETH3D/train/plant_scene_1', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [240, 320], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': False, 'pixelformer': False}


In [5]:
def droid_track(pixelformer_depth_folder = 'pixelformer_depth'):
    print("params:",vars(args))
    tstamps = []
    for (t, image, depth, intrinsics) in tqdm(image_stream(args.datapath, use_depth=True, stride=stride, use_pred_depth = args.pixelformer, pixelformer_depth_folder = pixelformer_depth_folder)):
        if not args.disable_vis:
            show_image(image[0])
    
        if t == 0:
            args.image_size = [image.shape[2], image.shape[3]]
            droid = Droid(args)
        if args.without_depth:
            
            droid.track(t, image, intrinsics=intrinsics)
        else:
            # print("t = ",t)    
            droid.track(t, image, depth, intrinsics=intrinsics)
    
    traj_est = droid.terminate(image_stream(args.datapath, use_depth=False, stride=stride))
    return traj_est

In [6]:
def evaluate_trajectory(traj_est):
    print("#"*20 + " Results...")
    image_path = os.path.join(args.datapath, 'rgb')
    images_list = sorted(glob.glob(os.path.join(image_path, '*.png')))[::stride]
    tstamps = [float(x.split('/')[-1][:-4]) for x in images_list]
    
    traj_est = PoseTrajectory3D(
        positions_xyz=traj_est[:,:3],
        orientations_quat_wxyz=traj_est[:,3:],
        timestamps=np.array(tstamps))
    
    gt_file = os.path.join(args.datapath, 'groundtruth.txt')
    traj_ref = file_interface.read_tum_trajectory_file(gt_file)
    
    traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est)
    
    result = main_ape.ape(traj_ref, traj_est, est_name='traj', 
        pose_relation=PoseRelation.translation_part, align=True, correct_scale=True) #originally correct_scale was False
    
    print(result.stats)
    print(result)
def save_traj(traj):
    os.makedirs("saved_traj_estimates",exist_ok=True)
    filename = args.datapath.split("/")[-1] + "_without_depth-" + str(args.without_depth) + "_pixel-" + str(args.pixelformer)
    save_path = os.path.join("saved_traj_estimates",filename)
    np.save(save_path, traj)
    print("trajectory saved as:", filename+".npy")
    return save_path+".npy"
    

In [7]:
def plot_trajectory(save_path):
    traj_est = np.load(save_path)
    translation_est = traj_est[:,:3]
    
    gt_file = os.path.join(args.datapath, 'groundtruth.txt')
    traj_ref = file_interface.read_tum_trajectory_file(gt_file)
    translation_gt = traj_ref.positions_xyz

    fig,ax = plt.subplots(1,2,figsize=(12,7))
    plt.suptitle('ETH-3D Dataset')
    ax[0].plot(translation_est[:,0],translation_est[:,1], label = "Estimated Traj")
    ax[0].legend()
    ax[1].plot(translation_gt[:,0],translation_gt[:,1], label = "GT Traj")
    ax[1].legend()
    plt.show()
    

In [58]:
# params to change
args.datapath = "../datasets/ETH3D/train/cables_1"
args.without_depth = False
args.pixelformer = False

traj_est = droid_track()
save_path = save_traj(traj_est)
evaluate_trajectory(traj_est)

params: {'datapath': '../datasets/ETH3D/train/cables_1', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [240, 320], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': False, 'pixelformer': False}


0it [00:00, ?it/s]

1180 1180
../droid.pth


1180it [01:15, 15.66it/s]


################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
Global BA Iteration #8
Global BA Iteration #9
Global BA Iteration #10
Global BA Iteration #11
Global BA Iteration #12
1180 1180
trajectory saved as: cables_1_without_depth-False_pixel-False.npy
#################### Results...
{'rmse': 0.007509604244555598, 'mean': 0.007082647340240851, 'median': 0.00725747650105077, 'std': 0.0024960493912634498, 'min': 0.000818154093833898, 'max': 0.014809809787479363, 'sse': 0.0664323156618003}
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	0.014810
      mean	0.007083
    median	0.007257
       min	0.000818
      rmse	0.007510
       sse	0.0

In [ ]:
print(traj_est.shape)

In [ ]:
plot_trajectory(save_path)

In [ ]:
print("helo")

In [56]:
# params to change
args.datapath = "../datasets/ETH3D/train/sfm_house_loop"
args.without_depth = False
args.pixelformer = True

traj_est = droid_track(pixelformer_depth_folder = 'pixelformer_depth')
save_traj(traj_est)
evaluate_trajectory(traj_est)

params: {'datapath': '../datasets/ETH3D/train/sfm_house_loop', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [240, 320], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': False, 'pixelformer': True}


0it [00:00, ?it/s]

556 556
initialising depth with 0
../droid.pth


556it [00:56,  9.76it/s]


################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
Global BA Iteration #8
Global BA Iteration #9
Global BA Iteration #10
Global BA Iteration #11
Global BA Iteration #12
556 556
#################### Results...
{'rmse': 0.9055769436950657, 'mean': 0.6987885443714965, 'median': 0.45451882392397175, 'std': 0.5759897318592245, 'min': 0.10234111085056768, 'max': 2.357951934089896, 'sse': 455.95869812936553}
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	2.357952
      mean	0.698789
    median	0.454519
       min	0.102341
      rmse	0.905577
       sse	455.958698
       std	0.575990



In [57]:
# params to change
args.datapath = "../datasets/ETH3D/train/sfm_house_loop"
args.without_depth = True
args.pixelformer = False

traj_est = droid_track()
save_traj(traj_est)
evaluate_trajectory(traj_est)

params: {'datapath': '../datasets/ETH3D/train/sfm_house_loop', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [344, 560], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': True, 'pixelformer': False}


0it [00:00, ?it/s]

556 556
../droid.pth


556it [00:58,  9.47it/s]


################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
Global BA Iteration #8
Global BA Iteration #9
Global BA Iteration #10
Global BA Iteration #11
Global BA Iteration #12
556 556
#################### Results...
{'rmse': 1.8065393121814914, 'mean': 1.3517021817749861, 'median': 1.2135431081401358, 'std': 1.1985347296769997, 'min': 0.07059018017548883, 'max': 4.525181218202944, 'sse': 1814.55286327019}
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	4.525181
      mean	1.351702
    median	1.213543
       min	0.070590
      rmse	1.806539
       sse	1814.552863
       std	1.198535



In [34]:
# params to change
args.datapath = "../datasets/ETH3D/train/table_3_long"
args.without_depth = True
args.pixelformer = False

traj_est = droid_track()
save_traj(traj_est)
evaluate_trajectory(traj_est)

params: {'datapath': '../datasets/ETH3D/train/table_3_long', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [240, 320], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': True, 'pixelformer': False}


0it [00:00, ?it/s]

1180 1180
../droid.pth


1180it [00:56, 20.96it/s]


################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
Global BA Iteration #8
Global BA Iteration #9
Global BA Iteration #10
Global BA Iteration #11
Global BA Iteration #12
1180 1180
trajectory saved as: table_3_long_without_depth-True_pixel-False.npy
#################### Results...
{'rmse': 0.00715659123219198, 'mean': 0.0063188272657054585, 'median': 0.0053520903358363105, 'std': 0.0033599434594743425, 'min': 0.0005384162464952738, 'max': 0.01658057304588858, 'sse': 0.04261237598981969}
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	0.016581
      mean	0.006319
    median	0.005352
       min	0.000538
      rmse	0.007157
       s

In [35]:
# params to change
args.datapath = "../datasets/ETH3D/train/table_3"
args.without_depth = True
args.pixelformer = False

traj_est = droid_track()
save_traj(traj_est)
evaluate_trajectory(traj_est)

params: {'datapath': '../datasets/ETH3D/train/table_3', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [344, 560], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': True, 'pixelformer': False}


0it [00:00, ?it/s]

1180 1180
../droid.pth


1180it [00:51, 22.78it/s]


################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
Global BA Iteration #8
Global BA Iteration #9
Global BA Iteration #10
Global BA Iteration #11
Global BA Iteration #12
1180 1180
trajectory saved as: table_3_without_depth-True_pixel-False.npy
#################### Results...
{'rmse': 0.007029612340294209, 'mean': 0.006268652791011161, 'median': 0.005539875413187815, 'std': 0.003181107014950082, 'min': 0.000633829455938819, 'max': 0.015877415527288764, 'sse': 0.04116306956246225}
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	0.015877
      mean	0.006269
    median	0.005540
       min	0.000634
      rmse	0.007030
       sse	0.04

In [12]:
# params to change
args.datapath = "../datasets/ETH3D/train/table_3_long_fb"
args.without_depth = True
args.pixelformer = False

traj_est = droid_track()
save_traj(traj_est)
evaluate_trajectory(traj_est)

params: {'datapath': '../datasets/ETH3D/train/table_3_long_fb', 'weights': '../droid.pth', 'buffer': 1024, 'image_size': [344, 560], 'disable_vis': True, 'beta': 0.5, 'filter_thresh': 2.0, 'warmup': 8, 'keyframe_thresh': 3.5, 'frontend_thresh': 16.0, 'frontend_window': 16, 'frontend_radius': 1, 'frontend_nms': 0, 'stereo': False, 'depth': False, 'backend_thresh': 22.0, 'backend_radius': 2, 'backend_nms': 3, 'upsample': False, 'reconstruction_path': '../saved_reconstruction', 'without_depth': True, 'pixelformer': False}


0it [00:00, ?it/s]

2359 2359
../droid.pth


2359it [01:51, 21.19it/s]


################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
################################
Global BA Iteration #1
Global BA Iteration #2
Global BA Iteration #3
Global BA Iteration #4
Global BA Iteration #5
Global BA Iteration #6
Global BA Iteration #7
Global BA Iteration #8
Global BA Iteration #9
Global BA Iteration #10
Global BA Iteration #11
Global BA Iteration #12
2359 2359
trajectory saved as: table_3_long_fb_without_depth-True_pixel-False.npy
#################### Results...
{'rmse': 0.0070611635888740325, 'mean': 0.006233421778473209, 'median': 0.0051789494077711514, 'std': 0.003317300734105412, 'min': 0.0008581114866220729, 'max': 0.016112952831418406, 'sse': 0.04153340601362406}
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	0.016113
      mean	0.006233
    median	0.005179
       min	0.000858
      rmse	0.007061
    

# FBFB Experiment to check in long sequences.

flowchart for FBFBFB experiment: 

1. make copy of seq folder : `cp -r table_3 table_3_long`
	and delete redundant files with command:
``` bash
cd table_3_long
rm -rf extrinsics_1_2.txt calibration2.txt sequence_calibration.txt rgb2 depth.txt rgb.txt imu*
```

till now
``` bash
seq=cables_1
cp -r $seq ${seq}_long
cd ${seq}_long
rm -rf extrinsics_1_2.txt calibration2.txt sequence_calibration.txt rgb2 depth.txt rgb.txt imu*

```
2. run cell [### removing redundant poses in gt.txt file.]
3. replace gt file with new one 
(already inside ${seq}_long )
``` bash 
rm groundtruth.txt
mv groundtruth_long.txt groundtruth.txt

```

3. make multiple directories for fb, fbf, fbfb : 
``` bash 
cd ..
cp -r ${seq}_long ${seq}_long_fb
cp -r ${seq}_long ${seq}_long_fbf
cp -r ${seq}_long ${seq}_long_fbfb

```

4. run cell for `fb`, then for `fbf`.....

5. replace gt.txt and assoc.txt with new one and delete original one.
``` bash 
seq=table_3
rm ${seq}_long_fb/groundtruth.txt ${seq}_long_fb/associated.txt
mv ${seq}_long_fb/groundtruth_long_fb.txt ${seq}_long_fb/groundtruth.txt
mv ${seq}_long_fb/associated_long_fb.txt ${seq}_long_fb/associated.txt

rm ${seq}_long_fbf/groundtruth.txt ${seq}_long_fbf/associated.txt
mv ${seq}_long_fbf/groundtruth_long_fbf.txt ${seq}_long_fbf/groundtruth.txt
mv ${seq}_long_fbf/associated_long_fbf.txt ${seq}_long_fbf/associated.txt

rm ${seq}_long_fbfb/groundtruth.txt ${seq}_long_fbfb/associated.txt
mv ${seq}_long_fbfb/groundtruth_long_fbfb.txt ${seq}_long_fbfb/groundtruth.txt
mv ${seq}_long_fbfb/associated_long_fbfb.txt ${seq}_long_fbfb/associated.txt

```


**misc**:
To start for a seq from scatch(delete all)
``` bash 
seq=cables_1
rm -r ${seq}_long ${seq}_long_fb ${seq}_long_fbf ${seq}_long_fbfb
```


### removing redundant poses in gt.txt file.

In [9]:
seq="cables_2"
dir_path = f"/home/suraj/scratch/DROID-SLAM/datasets/ETH3D/train/{seq}_long"
with open(os.path.join(dir_path,"groundtruth.txt")) as gt:
    with open(os.path.join(dir_path,"associated.txt")) as assoc:
        #read all assoctiations timestamps first (we need only those tstamps)
            
        assoc_lines = assoc.readlines()
        print("total frames available: ",len(assoc_lines))
        # make a dictionary out of gt.txt
        gt_lines = gt.readlines()
        gt_dict = {}
        for line in gt_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            gt_dict[float(line[0])] = line[1:]
        gt_tstamps = np.array(list(gt_dict.keys()))
        #print(gt_tstamps[0:10])
        
        #now for every image in assoc.txt, find it's corresponding pose and add it to the gt.txt
        # only if it is less than max_diff(=.01) apart in timestamps.
        max_diff = .01
        with open (os.path.join(dir_path,"groundtruth_long.txt"),"w") as gt_long:
            gt_long.write(gt_lines[0]) # adding commented first line
            poses_available = 0
            for line in assoc_lines:
                line_list = line.strip().split(" ")
                #print(truncate(float(line_list[0]),2))
                required_tstamp = float(line_list[0])
                diff_list = np.abs(gt_tstamps-required_tstamp)
                diffs = np.abs(gt_tstamps-required_tstamp)
                index = int(np.argmin(diffs))
                if diffs[index] <= max_diff:
                    """add pose for that image,depth pair in gt.txt"""
                    closest_tstamp = gt_tstamps[index]
                    #print(f"closest tstamp to {required_tstamp} in gt is {closest_tstamp}")
                    poses_available += 1
                    print_line = " ".join(gt_dict[closest_tstamp])
                    print_line = str(f"{closest_tstamp:.9f}") + " " + print_line + "\n"
                    gt_long.write(print_line)
                    #print(print_line)
                else:
                    print(f"pose not available for timestamp: {required_tstamp}")
                    gt_long.write("#not available\n")
            print("poses_available =",poses_available)

            

total frames available:  98
pose not available for timestamp: 11876.422479391
poses_available = 97


### generating long seqeuences manually by FB (forward-backward)

In [76]:
import copy
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)
seq = "sofa_1"
dir_path = f"/home/suraj/scratch/DROID-SLAM/datasets/ETH3D/train/{seq}_long_fb"
with open(os.path.join(dir_path,"groundtruth.txt")) as gt:
    with open(os.path.join(dir_path,"associated.txt")) as assoc:
        """ making gt.txt"""
        print('"""" making gt.txt"""')
        # make a dictionary out of gt.txt
        gt_lines = gt.readlines()
        print("len(gt_lines) = ",len(gt_lines))
        #make gt dict
        gt_dict = {}
        for line in gt_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            gt_dict[float(line[0])] = line[1:]
        gt_tstamps = np.array(list(gt_dict.keys()))
        #print(gt_tstamps[0:10])

        last_ind = len(gt_tstamps) - 1
        print("last_ind = ",last_ind)
        last_tstamp = gt_tstamps[last_ind]
        
        curr_ind = last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = gt_tstamps[curr_ind]
            curr_tstamp = last_tstamp + (last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            gt_dict[curr_tstamp] = gt_dict[corresponding_tstamp]
            gt_tstamps = np.append(gt_tstamps,curr_tstamp)
            curr_ind -= 1
        n = last_ind + 1
        
        print(f"verify: n = {n}, 2(n-1)+1 = {2*n-1} , tstamps now = {len(gt_tstamps)}")
        
        #now for every entry of gt_dict write its entry in gt_long.txt
        with open (os.path.join(dir_path,"groundtruth_long_fb.txt"),"w") as gt_long:
            for tstamp in gt_tstamps:
                
                print_line = " ".join(gt_dict[tstamp])
                print_line = str(f"{tstamp:.9f}") + " " + print_line + "\n"
                gt_long.write(print_line)
                #print(print_line)
        print("written gt.txt !!")

        
        """ making association.txt """
        print('\n\n""" making association.txt """')
        assoc_lines = assoc.readlines()
        print("len(assoc_lines) = ",len(assoc_lines))
        
        #make assoc dict
        assoc_dict = {}
        for line in assoc_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            assoc_dict[float(line[0])] = line[1:]
        assoc_tstamps = np.array(list(assoc_dict.keys()))
        #print(gt_tstamps[0:10])

        last_ind = len(assoc_tstamps) - 1
        print("last_ind = ",last_ind)
        last_tstamp = assoc_tstamps[last_ind]
        
        curr_ind = last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = assoc_tstamps[curr_ind]
            curr_tstamp = last_tstamp + (last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            
            assoc_dict[curr_tstamp] = copy.deepcopy(assoc_dict[corresponding_tstamp])
            
            new_img_ind = truncate(curr_tstamp,6)
            #copy rgb and depth images
            cmd_1 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][0]}") + " " + os.path.join(dir_path,f"rgb/{new_img_ind:.6f}.png")
            cmd_2 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][2]}") + " " + os.path.join(dir_path,f"depth/{new_img_ind:.6f}.png")
            cmd_3 = "cp " + os.path.join(dir_path,f"pixelformer_depth/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth/{new_img_ind:.6f}.png")
            cmd_4 = "cp " + os.path.join(dir_path,f"pixelformer_depth10/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth10/{new_img_ind:.6f}.png")
            os.system(cmd_1)
            os.system(cmd_2)
            os.system(cmd_3)
            os.system(cmd_4)
            
            
            assoc_dict[curr_tstamp][1] = f"{curr_tstamp:.9f}"
            assoc_dict[curr_tstamp][0] = f"rgb/{new_img_ind:.6f}.png"
            assoc_dict[curr_tstamp][2] = f"depth/{new_img_ind:.6f}.png"
            assoc_tstamps = np.append(assoc_tstamps,curr_tstamp)
            curr_ind -= 1
        n = last_ind + 1
        
        print(f"verify: n = {n}, 2(n-1)+1 = {2*n-1} , tstamps now = {len(assoc_tstamps)}")
        
        #now for every entry of gt_dict write its entry in gt_long.txt
        with open (os.path.join(dir_path,"associated_long_fb.txt"),"w") as assoc_long:
            for tstamp in assoc_tstamps:
                
                print_line = " ".join(assoc_dict[tstamp])
                print_line = str(f"{tstamp:.9f}") + " " + print_line + "\n"
                assoc_long.write(print_line)
                #print(print_line)
        print(f"written assoc.txt for seq = {seq} !!")

"""" making gt.txt"""
len(gt_lines) =  765
last_ind =  763
verify: n = 764, 2(n-1)+1 = 1527 , tstamps now = 1527
written gt.txt !!


""" making association.txt """
len(assoc_lines) =  976
last_ind =  975
verify: n = 976, 2(n-1)+1 = 1951 , tstamps now = 1951
written assoc.txt for seq = sofa_1 !!


### generating long seqeuences manually by FBF

In [77]:
import copy
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)
#seq = "table_3"
dir_path = f"/home/suraj/scratch/DROID-SLAM/datasets/ETH3D/train/{seq}_long_fbf"
with open(os.path.join(dir_path,"groundtruth.txt")) as gt:
    with open(os.path.join(dir_path,"associated.txt")) as assoc:
        """ making gt.txt"""
        print('"""" making gt.txt"""')
        # make a dictionary out of gt.txt
        gt_lines = gt.readlines()
        print("len(gt_lines) = ",len(gt_lines))
        #make gt dict
        gt_dict = {}
        for line in gt_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            gt_dict[float(line[0])] = line[1:]
        gt_tstamps = np.array(list(gt_dict.keys()))
        #print(gt_tstamps[0:10])

        last_ind = len(gt_tstamps) - 1
        print("last_ind = ",last_ind)
        last_tstamp = gt_tstamps[last_ind]
        
        curr_ind = last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = gt_tstamps[curr_ind]
            curr_tstamp = last_tstamp + (last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            
            gt_dict[curr_tstamp] = gt_dict[corresponding_tstamp]
            gt_tstamps = np.append(gt_tstamps,curr_tstamp)
            curr_ind -= 1
        n = last_ind + 1 # original no. entries
        print(f"verify FB: n = {n}, 2(n-1)+1 = {2*n-1} , tstamps now = {len(gt_tstamps)}")
        
        fb_last_ind = len(gt_tstamps) - 1
        print("fb_last_ind = ",fb_last_ind)
        fb_last_tstamp = gt_tstamps[fb_last_ind]
        curr_ind = fb_last_ind - 1
        while curr_ind >= last_ind:
            corresponding_tstamp = gt_tstamps[curr_ind]
            curr_tstamp = fb_last_tstamp + (fb_last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            gt_dict[curr_tstamp] = gt_dict[corresponding_tstamp]
            gt_tstamps = np.append(gt_tstamps,curr_tstamp)
            curr_ind -= 1
        
        print(f"verify FBF: n = {n}, 3(n-1)+1 = {3*n-2} , tstamps now = {len(gt_tstamps)}")
        
        #now for every entry of gt_dict write its entry in gt_long.txt
        with open (os.path.join(dir_path,"groundtruth_long_fbf.txt"),"w") as gt_long:
            for tstamp in gt_tstamps:
                
                print_line = " ".join(gt_dict[tstamp])
                print_line = str(f"{tstamp:.9f}") + " " + print_line + "\n"
                gt_long.write(print_line)
                #print(print_line)
        print("written gt.txt !!")
        
        
        """ making association.txt """
        
        print('\n\n""" making association.txt """')
        assoc_lines = assoc.readlines()
        print("len(assoc_lines) = ",len(assoc_lines))
        
        #make assoc dict
        assoc_dict = {}
        for line in assoc_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            assoc_dict[float(line[0])] = line[1:]
        assoc_tstamps = np.array(list(assoc_dict.keys()))
        #print(gt_tstamps[0:10])

        last_ind = len(assoc_tstamps) - 1
        print("last_ind = ",last_ind)
        last_tstamp = assoc_tstamps[last_ind]
        
        curr_ind = last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = assoc_tstamps[curr_ind]
            curr_tstamp = last_tstamp + (last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            
            assoc_dict[curr_tstamp] = copy.deepcopy(assoc_dict[corresponding_tstamp])
            new_img_ind = truncate(curr_tstamp,6)
            
            #copy rgb and depth images
            cmd_1 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][0]}") + " " + os.path.join(dir_path,f"rgb/{new_img_ind:.6f}.png")
            cmd_2 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][2]}") + " " + os.path.join(dir_path,f"depth/{new_img_ind:.6f}.png")
            cmd_3 = "cp " + os.path.join(dir_path,f"pixelformer_depth/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth/{new_img_ind:.6f}.png")
            cmd_4 = "cp " + os.path.join(dir_path,f"pixelformer_depth10/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth10/{new_img_ind:.6f}.png")
            os.system(cmd_1)
            os.system(cmd_2)
            os.system(cmd_3)
            os.system(cmd_4)
            
            assoc_dict[curr_tstamp][1] = f"{curr_tstamp:.9f}"
            assoc_dict[curr_tstamp][0] = f"rgb/{new_img_ind:.6f}.png"
            assoc_dict[curr_tstamp][2] = f"depth/{new_img_ind:.6f}.png"
            assoc_tstamps = np.append(assoc_tstamps,curr_tstamp)
            curr_ind -= 1
        n = last_ind + 1
        print(f"verify: n = {n}, 2(n-1)+1 = {2*n-1} , tstamps now = {len(assoc_tstamps)}")
        
        fb_last_ind = len(assoc_tstamps) - 1
        print("fb_last_ind = ",fb_last_ind)
        fb_last_tstamp = gt_tstamps[fb_last_ind]
        curr_ind = fb_last_ind - 1
        while curr_ind >= last_ind:
            corresponding_tstamp = assoc_tstamps[curr_ind]
            curr_tstamp = fb_last_tstamp + (fb_last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            assoc_dict[curr_tstamp] = copy.deepcopy(assoc_dict[corresponding_tstamp])
            new_img_ind = truncate(curr_tstamp,6)
            
            #copy rgb and depth images
            cmd_1 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][0]}") + " " + os.path.join(dir_path,f"rgb/{new_img_ind:.6f}.png")
            cmd_2 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][2]}") + " " + os.path.join(dir_path,f"depth/{new_img_ind:.6f}.png")
            cmd_3 = "cp " + os.path.join(dir_path,f"pixelformer_depth/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth/{new_img_ind:.6f}.png")
            cmd_4 = "cp " + os.path.join(dir_path,f"pixelformer_depth10/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth10/{new_img_ind:.6f}.png")
            os.system(cmd_1)
            os.system(cmd_2)
            os.system(cmd_3)
            os.system(cmd_4)
            
            assoc_dict[curr_tstamp][1] = f"{curr_tstamp:.9f}"
            assoc_dict[curr_tstamp][0] = f"rgb/{new_img_ind:.6f}.png"
            assoc_dict[curr_tstamp][2] = f"depth/{new_img_ind:.6f}.png"
            assoc_tstamps = np.append(assoc_tstamps,curr_tstamp)
            curr_ind -= 1
        print(f"verify FBF: n = {n}, 3(n-1)+1 = {3*n-2} , tstamps now = {len(assoc_tstamps)}")
        
        #now for every entry of gt_dict write its entry in gt_long.txt
        with open (os.path.join(dir_path,"associated_long_fbf.txt"),"w") as assoc_long:
            for tstamp in assoc_tstamps:
                
                print_line = " ".join(assoc_dict[tstamp])
                print_line = str(f"{tstamp:.9f}") + " " + print_line + "\n"
                assoc_long.write(print_line)
                #print(print_line)
        print(f"written assoc.txt for seq = {seq}!!")
        

"""" making gt.txt"""
len(gt_lines) =  765
last_ind =  763
verify FB: n = 764, 2(n-1)+1 = 1527 , tstamps now = 1527
fb_last_ind =  1526
verify FBF: n = 764, 3(n-1)+1 = 2290 , tstamps now = 2290
written gt.txt !!


""" making association.txt """
len(assoc_lines) =  976
last_ind =  975
verify: n = 976, 2(n-1)+1 = 1951 , tstamps now = 1951
fb_last_ind =  1950
verify FBF: n = 976, 3(n-1)+1 = 2926 , tstamps now = 2926
written assoc.txt for seq = sofa_1!!


### generating long seqeuences manually by FBFB

In [78]:
import copy
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)
#seq="table_3"
dir_path = f"/home/suraj/scratch/DROID-SLAM/datasets/ETH3D/train/{seq}_long_fbfb"
with open(os.path.join(dir_path,"groundtruth.txt")) as gt:
    with open(os.path.join(dir_path,"associated.txt")) as assoc:
        """ making gt.txt"""
        print('"""" making gt.txt"""')
        # make a dictionary out of gt.txt
        gt_lines = gt.readlines()
        print("len(gt_lines) = ",len(gt_lines))
        #make gt dict
        gt_dict = {}
        for line in gt_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            gt_dict[float(line[0])] = line[1:]
        gt_tstamps = np.array(list(gt_dict.keys()))
        #print(gt_tstamps[0:10])

        last_ind = len(gt_tstamps) - 1
        print("last_ind = ",last_ind)
        last_tstamp = gt_tstamps[last_ind]
        
        curr_ind = last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = gt_tstamps[curr_ind]
            curr_tstamp = last_tstamp + (last_tstamp - corresponding_tstamp)
            gt_dict[curr_tstamp] = gt_dict[corresponding_tstamp]
            gt_tstamps = np.append(gt_tstamps,curr_tstamp)
            curr_ind -= 1
        n = last_ind + 1 # original no. entries
        print(f"verify FB: n = {n}, 2(n-1)+1 = {2*n-1} , tstamps now = {len(gt_tstamps)}")
        
        fb_last_ind = len(gt_tstamps) - 1
        print("fb_last_ind = ",fb_last_ind)
        fb_last_tstamp = gt_tstamps[fb_last_ind]
        curr_ind = fb_last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = gt_tstamps[curr_ind]
            curr_tstamp = fb_last_tstamp + (fb_last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            gt_dict[curr_tstamp] = gt_dict[corresponding_tstamp]
            gt_tstamps = np.append(gt_tstamps,curr_tstamp)
            curr_ind -= 1
        
        print(f"verify FBF: n = {n}, 4(n-1)+1 = {4*(n-1)+1} , tstamps now = {len(gt_tstamps)}")
        
        #now for every entry of gt_dict write its entry in gt_long.txt
        with open (os.path.join(dir_path,"groundtruth_long_fbfb.txt"),"w") as gt_long:
            for tstamp in gt_tstamps:
                
                print_line = " ".join(gt_dict[tstamp])
                print_line = str(f"{tstamp:.9f}") + " " + print_line + "\n"
                gt_long.write(print_line)
                #print(print_line)
        print("written gt.txt !!")
        
        
        """ making association.txt """
        
        print('\n\n""" making association.txt """')
        assoc_lines = assoc.readlines()
        print("len(assoc_lines) = ",len(assoc_lines))
        
        #make assoc dict
        assoc_dict = {}
        for line in assoc_lines:
            if line.startswith("#"):
                continue
            line = line.strip().split(" ")
            assoc_dict[float(line[0])] = line[1:]
        assoc_tstamps = np.array(list(assoc_dict.keys()))
        #print(gt_tstamps[0:10])

        last_ind = len(assoc_tstamps) - 1
        print("last_ind = ",last_ind)
        last_tstamp = assoc_tstamps[last_ind]
        
        curr_ind = last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = assoc_tstamps[curr_ind]
            curr_tstamp = last_tstamp + (last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            
            assoc_dict[curr_tstamp] = copy.deepcopy(assoc_dict[corresponding_tstamp])
            new_img_ind = truncate(curr_tstamp,6)
            
            #copy rgb and depth images
            cmd_1 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][0]}") + " " + os.path.join(dir_path,f"rgb/{new_img_ind:.6f}.png")
            cmd_2 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][2]}") + " " + os.path.join(dir_path,f"depth/{new_img_ind:.6f}.png")
            cmd_3 = "cp " + os.path.join(dir_path,f"pixelformer_depth/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth/{new_img_ind:.6f}.png")
            cmd_4 = "cp " + os.path.join(dir_path,f"pixelformer_depth10/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth10/{new_img_ind:.6f}.png")
            os.system(cmd_1)
            os.system(cmd_2)
            os.system(cmd_3)
            os.system(cmd_4)
            
            assoc_dict[curr_tstamp][1] = f"{curr_tstamp:.9f}"
            assoc_dict[curr_tstamp][0] = f"rgb/{new_img_ind:.6f}.png"
            assoc_dict[curr_tstamp][2] = f"depth/{new_img_ind:.6f}.png"
            assoc_tstamps = np.append(assoc_tstamps,curr_tstamp)
            curr_ind -= 1
        n = last_ind + 1
        print(f"verify: n = {n}, 2(n-1)+1 = {2*n-1} , tstamps now = {len(assoc_tstamps)}")
        
        fb_last_ind = len(assoc_tstamps) - 1
        print("fb_last_ind = ",fb_last_ind)
        fb_last_tstamp = gt_tstamps[fb_last_ind]
        curr_ind = fb_last_ind - 1
        while curr_ind >= 0:
            corresponding_tstamp = assoc_tstamps[curr_ind]
            curr_tstamp = fb_last_tstamp + (fb_last_tstamp - corresponding_tstamp)
            curr_tstamp = truncate(curr_tstamp,9)
            assoc_dict[curr_tstamp] = copy.deepcopy(assoc_dict[corresponding_tstamp])
            new_img_ind = truncate(curr_tstamp,6)
            
            #copy rgb and depth images
            cmd_1 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][0]}") + " " + os.path.join(dir_path,f"rgb/{new_img_ind:.6f}.png")
            cmd_2 = "cp " + os.path.join(dir_path,f"{assoc_dict[corresponding_tstamp][2]}") + " " + os.path.join(dir_path,f"depth/{new_img_ind:.6f}.png")
            cmd_3 = "cp " + os.path.join(dir_path,f"pixelformer_depth/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth/{new_img_ind:.6f}.png")
            cmd_4 = "cp " + os.path.join(dir_path,f"pixelformer_depth10/{truncate(float(assoc_dict[corresponding_tstamp][1]),6):.6f}.png") + " " + os.path.join(dir_path,f"pixelformer_depth10/{new_img_ind:.6f}.png")
            os.system(cmd_1)
            os.system(cmd_2)
            os.system(cmd_3)
            os.system(cmd_4)
            
            assoc_dict[curr_tstamp][1] = f"{curr_tstamp:.9f}"
            assoc_dict[curr_tstamp][0] = f"rgb/{new_img_ind:.6f}.png"
            assoc_dict[curr_tstamp][2] = f"depth/{new_img_ind:.6f}.png"
            assoc_tstamps = np.append(assoc_tstamps,curr_tstamp)
            curr_ind -= 1
        print(f"verify FBF: n = {n}, 4(n-1)+1 = {4*(n-1)+1} , tstamps now = {len(assoc_tstamps)}")
        
        #now for every entry of gt_dict write its entry in gt_long.txt
        with open (os.path.join(dir_path,"associated_long_fbfb.txt"),"w") as assoc_long:
            for tstamp in assoc_tstamps:
                
                print_line = " ".join(assoc_dict[tstamp])
                print_line = str(f"{tstamp:.9f}") + " " + print_line + "\n"
                assoc_long.write(print_line)
                #print(print_line)
        print(f"written assoc.txt for seq = {seq} !!")
        

"""" making gt.txt"""
len(gt_lines) =  765
last_ind =  763
verify FB: n = 764, 2(n-1)+1 = 1527 , tstamps now = 1527
fb_last_ind =  1526
verify FBF: n = 764, 4(n-1)+1 = 3053 , tstamps now = 3053
written gt.txt !!


""" making association.txt """
len(assoc_lines) =  976
last_ind =  975
verify: n = 976, 2(n-1)+1 = 1951 , tstamps now = 1951
fb_last_ind =  1950
verify FBF: n = 976, 4(n-1)+1 = 3901 , tstamps now = 3901
written assoc.txt for seq = sofa_1 !!
